In [91]:
import pyautogui
import cv2
import math
import numpy as np # numpy 패키지 로드하여 np로 사용
import torch
from PIL import ImageGrab
import pytesseract
from ultralytics import YOLO

In [92]:
#setup
webcamYN = False
pyautoguiYn = False
imageGrabYn = True

textYN = True
PytesseractYN = True

#yolo version select
yolov5 = False
yolov8 = False
yolov10 = True

#이미지 사이즈 관련
# 캡쳐 시작 좌표, 가로, 세로 길이 4k 모니터에서 좌측 유튜브 영상 기준
cX, cY, cWidth, cHeight = 10, 195, 1890, 1000
#이미지 리사이즈 크기
n = 3
newSizeX = int((cWidth - cX) / n)
newSizeY = int((cHeight - cY) / n)
new_size = (newSizeX, newSizeY)

model = ""
title = ""

In [93]:
#yolov5 세팅
# 모델 로드 (pre-trained 모델 사용) 사람 차량등 사물 인식
if yolov5:
    model = torch.hub.load('ultralytics/yolov5', 'yolov5n')  # 'yolov5s', 'yolov5m', 'yolov5l', 'yolov5x' 가능
    title = "YOLOv5"
    getFinded = lambda img : model(img).render()[0]

    # YOLO 모델을 사용해 객체 탐지
    #results = model(img)

    #결과 이미지를 얻고, 원본 BGR 형식으로 다시 변환
    #img = results.render()[0]  # render()는 탐지된 객체에 박스를 그린 이미지를 반환합니다.

In [94]:
#yolov8 세팅
if yolov8:
    model = YOLO('yolov8n.pt')
    title = "YOLOv8"
    getFinded = lambda img : model(img)[0].plot()

In [95]:
#yolov10 세팅
if yolov10:
    model = YOLO('yolov10n.pt')
    title = "YOLOv10"
    getFinded = lambda img : model(img)[0].plot()

In [96]:
# 캡쳐 시작 좌표, 가로, 세로 길이 4k 모니터에서 좌측 유튜브 영상 기준
#x, y, width, height = 10, 195, 1890, 1000

#윈도우 화면 캡쳐
def monitorCapture(x, y, width, height):
    #im = pyautogui.screenshot(region=(x, y, width, height))
    im = ImageGrab.grab(bbox=(x, y, x + width, y + height)) #이미지 캡쳐
    im = im.convert('L') #이미지 흑백
    im = im.resize(new_size)
    return cv2.cvtColor(np.array(im), cv2.COLOR_BGR2RGB)



In [97]:
#웹캠 설정
global webcamItem
def setupToWebcam():
    webcamItem = cv2.VideoCapture(0)

    # 웹캠이 열리지 않을 경우
    if not webcamItem.isOpened():
        print("Error: Could not open webcam.")
        exit()

def webcam():
    ret, frame = webcamItem.read()

    if not ret:
        print("Error: Could not read frame.")

    # 프레임 크기를 YOLO 모델에 맞게 조정
    return cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # 모델은 RGB 형식의 이미지를 사용합니다.

def webcalRelease():
  webcamItem.release()
if webcamYN:
    setupToWebcam()

In [98]:
def checkoToPytesseract(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
  data = pytesseract.image_to_data(gray, output_type=Output.DICT)

  # 추출된 텍스트 박스 그리기
  n_boxes = len(data['text'])
  for i in range(n_boxes):
    if int(data['conf'][i]) > 60:  # 신뢰도 기준으로 필터링
      (x, y, w, h) = (data['left'][i], data['top'][i], data['width'][i], data['height'][i])
      cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
      cv2.putText(image, data['text'][i], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

In [99]:
while True:
  #캐니 엣지 값
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break
  
  #이미지 캡쳐 사용시
  if imageGrabYn:
    img = monitorCapture(cX, cY, cWidth, cHeight)
    
  #웹캠 사용시시
  if webcamYN:
    img = webcam()
   
  # YOLO 모델을 사용해 객체 탐지
  #results = model(img)

  #결과 이미지를 얻고, 원본 BGR 형식으로 다시 변환
  #img = results.render()[0]  # render()는 탐지된 객체에 박스를 그린 이미지를 반환합니다.
  img = getFinded(img)
  
  #if PytesseractYN:
  #  checkoToPytesseract(img)
  # 화면에 결과 표시
  cv2.imshow(title + ' Real-time Detection', img)



0: 288x640 1 person, 86.7ms
Speed: 2.0ms preprocess, 86.7ms inference, 0.0ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 (no detections), 83.2ms
Speed: 1.0ms preprocess, 83.2ms inference, 1.0ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 person, 79.1ms
Speed: 2.0ms preprocess, 79.1ms inference, 0.0ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 2 persons, 82.7ms
Speed: 2.0ms preprocess, 82.7ms inference, 0.0ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 person, 81.2ms
Speed: 1.0ms preprocess, 81.2ms inference, 0.0ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 bird, 85.2ms
Speed: 1.0ms preprocess, 85.2ms inference, 0.0ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 person, 76.6ms
Speed: 2.0ms preprocess, 76.6ms inference, 0.0ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 (no detections), 81.2ms
Speed: 1.5ms preprocess, 81.2ms inference, 1.0ms postprocess per image at sh

In [100]:
# 리소스 해제
if webcamYN:
  webcalRelease
cv2.destroyAllWindows()